## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

(50000, 32, 32, 3)


In [3]:
## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


### 試驗一

In [8]:
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3),padding='same', activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.3563 - accuracy: 0.5248
Epoch 2/100
50000/50000 [==============================] - 5s 107us/step - loss: 0.9469 - accuracy: 0.6670
Epoch 3/100
50000/50000 [==============================] - 5s 107us/step - loss: 0.7858 - accuracy: 0.7250
Epoch 4/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.6686 - accuracy: 0.7663
Epoch 5/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.5558 - accuracy: 0.8055
Epoch 6/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.4448 - accuracy: 0.8457
Epoch 7/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.3521 - accuracy: 0.8801
Epoch 8/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.2729 - accuracy: 0.9065
Epoch 9/100
50000/50000 [==============================] - 5s 107us/step - loss: 0.2037 - accuracy: 0.9287
Epoch 10/100
50000/50000 [===========

Epoch 77/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0213 - accuracy: 0.9933
Epoch 78/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0203 - accuracy: 0.9938
Epoch 79/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0168 - accuracy: 0.9944
Epoch 80/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0156 - accuracy: 0.9951
Epoch 81/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0208 - accuracy: 0.9936
Epoch 82/100
50000/50000 [==============================] - 5s 107us/step - loss: 0.0180 - accuracy: 0.9943
Epoch 83/100
50000/50000 [==============================] - 5s 110us/step - loss: 0.0203 - accuracy: 0.9937
Epoch 84/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0201 - accuracy: 0.9934
Epoch 85/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0132 - accuracy: 0.9959
Epoch 86/100
50000/50000 [==

In [9]:
# Score trained model.
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 97us/step
Test loss: 4.991688199996949
Test accuracy: 0.6549999713897705


## 試驗二

In [10]:
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',activation='relu'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 6s 123us/step - loss: 1.3333 - accuracy: 0.5294
Epoch 2/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.8651 - accuracy: 0.6950
Epoch 3/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.6663 - accuracy: 0.7661
Epoch 4/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.5278 - accuracy: 0.8138
Epoch 5/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.4058 - accuracy: 0.8579
Epoch 6/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.3080 - accuracy: 0.8927
Epoch 7/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.2180 - accuracy: 0.9251
Epoch 8/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.1651 - accuracy: 0.9412
Epoch 9/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.1271 - accuracy: 0.9558
Epoch 10/100
50000/50000 [===========

Epoch 77/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0168 - accuracy: 0.9949
Epoch 78/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0192 - accuracy: 0.9940
Epoch 79/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0153 - accuracy: 0.9955
Epoch 80/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0108 - accuracy: 0.9968
Epoch 81/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0140 - accuracy: 0.9959
Epoch 82/100
50000/50000 [==============================] - 5s 108us/step - loss: 0.0223 - accuracy: 0.9934
Epoch 83/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0167 - accuracy: 0.9944
Epoch 84/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0167 - accuracy: 0.9948
Epoch 85/100
50000/50000 [==============================] - 5s 109us/step - loss: 0.0090 - accuracy: 0.9973
Epoch 86/100
50000/50000 [==

In [11]:
# Score trained model.
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 98us/step
Test loss: 2.783994051900506
Test accuracy: 0.7605999708175659


## 試驗三

In [12]:
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',activation='relu'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#改成gloabal average pooling
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 6s 125us/step - loss: 1.3936 - accuracy: 0.4946
Epoch 2/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.9871 - accuracy: 0.6496
Epoch 3/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.8306 - accuracy: 0.7087
Epoch 4/100
50000/50000 [==============================] - 6s 110us/step - loss: 0.7278 - accuracy: 0.7447
Epoch 5/100
50000/50000 [==============================] - 6s 110us/step - loss: 0.6499 - accuracy: 0.7717
Epoch 6/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.5895 - accuracy: 0.7923
Epoch 7/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.5427 - accuracy: 0.8100
Epoch 8/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.4953 - accuracy: 0.8251
Epoch 9/100
50000/50000 [==============================] - 6s 110us/step - loss: 0.4624 - accuracy: 0.8386
Epoch 10/100
50000/50000 [===========

Epoch 77/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.0449 - accuracy: 0.9840
Epoch 78/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.0461 - accuracy: 0.9835
Epoch 79/100
50000/50000 [==============================] - 6s 112us/step - loss: 0.0452 - accuracy: 0.9840
Epoch 80/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.0415 - accuracy: 0.9849
Epoch 81/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.0401 - accuracy: 0.9858
Epoch 82/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.0475 - accuracy: 0.9834
Epoch 83/100
50000/50000 [==============================] - 6s 111us/step - loss: 0.0443 - accuracy: 0.9845
Epoch 84/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.0398 - accuracy: 0.9854
Epoch 85/100
50000/50000 [==============================] - 6s 114us/step - loss: 0.0388 - accuracy: 0.9868
Epoch 86/100
50000/50000 [==

In [13]:
# Score trained model.
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 93us/step
Test loss: 1.773426138973236
Test accuracy: 0.7577999830245972


## 試驗四

In [14]:
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same',activation='relu'))
# classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(Convolution2D(filters=64, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 5s 96us/step - loss: 1.2871 - accuracy: 0.5397
Epoch 2/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.8287 - accuracy: 0.7093
Epoch 3/100
50000/50000 [==============================] - 4s 85us/step - loss: 0.6570 - accuracy: 0.7709
Epoch 4/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.5354 - accuracy: 0.8131
Epoch 5/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.4274 - accuracy: 0.8492
Epoch 6/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.3264 - accuracy: 0.8866
Epoch 7/100
50000/50000 [==============================] - 4s 85us/step - loss: 0.2374 - accuracy: 0.9185
Epoch 8/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.1705 - accuracy: 0.9411
Epoch 9/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.1297 - accuracy: 0.9555
Epoch 10/100
50000/50000 [====================

50000/50000 [==============================] - 4s 86us/step - loss: 0.0169 - accuracy: 0.9944
Epoch 78/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0145 - accuracy: 0.9956
Epoch 79/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0150 - accuracy: 0.9953
Epoch 80/100
50000/50000 [==============================] - 4s 85us/step - loss: 0.0223 - accuracy: 0.9931
Epoch 81/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0142 - accuracy: 0.9955
Epoch 82/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0145 - accuracy: 0.9954
Epoch 83/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0111 - accuracy: 0.9964
Epoch 84/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0199 - accuracy: 0.9940
Epoch 85/100
50000/50000 [==============================] - 4s 86us/step - loss: 0.0132 - accuracy: 0.9956
Epoch 86/100
50000/50000 [========================

In [15]:
# Score trained model.
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 82us/step
Test loss: 2.874645990753174
Test accuracy: 0.7483000159263611


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)